# Алгоритм решения системы регулярных выражений
Пусть $\mathcal{K}$ — алгебра регулярных выражений на конечным алфавитом $\Sigma$ сигнатуры $\langle 0, 1, +, \cdot, ^* \rangle$. \
Будем решать системы вида:

$$
\begin{cases}
    \begin{gather*}
    X_1 = \alpha_{11}X_1 + ... + \alpha_{1n}X_n + \beta_i, \\
    ... \tag{1}\\
    X_n = \alpha_{n1}X_1 + ... + \alpha_{nn}X_n + \beta_n; \\
    \end{gather*}
\end{cases}
$$

где $\alpha_{ij}, \beta_{ij} \in \mathcal{K}$.

Для текущей задачи решение таких систем избыточно, так как синтаксис входных данных предполагает, что $\alpha_{ij} \neq 1, \beta_{i} \notin \{0; 1\}$ (это дает нам всегда единственное решение), однако решение систем в общем виде потребуется при переводе *DFA* в *regex*, так что сразу реализуем алгоритм, который решает системы и общего вида.

## Алгебра регулярных выражений

В связи с тем, что уравнения в системе придется решать в алгебре $\mathcal{K}$, было бы неплохо реализовать класс регулярных выражений с необходимыми операциями.

Так как $\forall R \in \mathcal{K}: R = (R)$, возникает проблема лишних скобок при приведении в строку регулярного выражения. Для решения данной проблемы экзмепляр класса можно представить в виде древовидной стуктуры, а при создании экземпляра (из строки) строить дерево разбора, согласно следующей\
грамматике $\mathcal{G}$:

$$
\begin{align*}
    S &\rightarrow T_1\textcolor{red}{\textbf{|}}S \; | \; T_1 \\
    T_1 &\rightarrow T_2T_1 \; | \; T_2 \\
    T_2 &\rightarrow T_3^{\textcolor{red}{*}} \; | \; T_3 \\
    T_3 &\rightarrow C \; | \; \textcolor{red}{(}S\textcolor{red}{)} \; | \; \varepsilon
\end{align*}
$$

где $C$ — символ из $\Sigma$ $(\Sigma \cap \{|, ^*, (, )\} = \varnothing)$.

### Класс регулярных выражений

In [142]:
import re
from enum import Enum, auto

class Operation(Enum):
    CONST = auto()
    ALT = auto()
    CONCAT = auto()
    KLEENE = auto()

ZERO = '0'
ONE = '1'

def parse_symbol(tokens):
    if re.fullmatch('[a-z01]', tokens[0]):
        return Regex(Operation.CONST, tokens.pop(0))

class Regex():
    def __init__(self, operation=None, operands=ZERO):
        global ONE, ZERO
        if operation == None:
            self.operation = Operation.CONST
            self.operands = ZERO
        elif operation == '':
            self.operation = Operation.CONST
            self.operands = ONE
        else:
            self.operation = operation
            self.operands = operands
    
    def parse(value):
        global parse_symbol
        tokens = [token for token in value] if isinstance(value, str) else value
        def parse_S():
            regex = parse_T1()
            if len(tokens) != 0 and tokens[0] == '|':
                tokens.pop(0)
                other_regex = parse_S()
                return regex + other_regex
            return regex
        def parse_T1():
            regex = parse_T2()
            if regex == Regex(''):
                return regex
            other_regex = parse_T1()
            if other_regex != Regex(''):
                return regex * other_regex
            return regex
        def parse_T2():
            regex = parse_T3()
            if regex == Regex(''):
                return regex
            if len(tokens) != 0 and tokens[0] == '*':
                tokens.pop(0)
                return regex.kleene_star()
            return regex
        def parse_T3():
            if len(tokens) == 0:
                return Regex('')
            regex = parse_symbol(tokens)
            if regex != None:
                return regex
            if tokens[0] == '(':
                tokens.pop(0)
                regex = parse_S()
                if len(tokens) != 0 and tokens.pop(0) == ')':
                    return regex
                raise Exception('Incorrect regex')
            return Regex('')
        regex = parse_S()
        if isinstance(value, str) and len(tokens) != 0:
            raise Exception('Incorrect regex')
        return regex
    
    def __eq__(self, regex):
        return isinstance(regex, Regex) and self.operation == regex.operation and self.operands == regex.operands
    
    def set_zero(value):
        global ZERO
        if value == None:
            return
        ZERO = value
    
    def set_one(value):
        global ONE
        if value == None:
            return
        ONE = value
    
    def set_parse_symbol(func):
        global parse_symbol
        if func == None:
            return
        parse_symbol = func
    
    def is_zero(self):
        global ZERO
        return self.operation == Operation.CONST and self.operands == ZERO
    
    def is_one(self):
        global ONE
        return self.operation == Operation.CONST and self.operands == ONE
    
    def kleene_star(regex):
        if regex.is_zero() or regex.is_one():
            return Regex('')
        if regex.operation == Operation.KLEENE:
            return regex
        return Regex(Operation.KLEENE, regex)

    def __add__(self, regex):
        if self.is_zero():
            return regex
        if regex.is_zero():
            return self 
        if self == regex:
            return self
        self_operands = [self]
        if self.operation == Operation.ALT:
            self_operands = self.operands
        regex_operands = [regex]
        if regex.operation == Operation.ALT:
            regex_operands = regex.operands
        operands = []
        for operand in self_operands + regex_operands:
            if operand not in operands:
                operands.append(operand)
        return Regex(Operation.ALT, operands)
    
    def __mul__(self, regex):
        if self.is_zero() or regex.is_zero():
            return Regex()
        if self.is_one():
            return regex
        if regex.is_one():
            return self
        self_operands = [self]
        if self.operation == Operation.CONCAT:
            self_operands = self.operands
        regex_operands = [regex]
        if regex.operation == Operation.CONCAT:
            regex_operands = regex.operands
        return Regex(Operation.CONCAT, self_operands + regex_operands)
    
    def __str__(self):
        global ONE
        match self.operation:
            case Operation.CONST:
                return self.operands
            case Operation.KLEENE:
                if self.operands.operation == Operation.CONST:
                    return f'{self.operands}*'
                return f'({self.operands})*'
            case Operation.CONCAT:
                result = ''
                for operand in self.operands:
                    if operand.operation == Operation.ALT:
                        result += f'({operand})'
                        continue
                    result +=  f'{operand}'
                return result
            case Operation.ALT:
                return '|'.join(map(str, self.operands)).replace(f'{ONE}', '')

## Уравнения

### Парсер

Так как грамматика входных данных не допускает линейные уравнения общего вида, а делать тесты и использовать решение систем в следующих задачах придется, построим грамматику, способную разбирать любые линейные уравнения:

$$
\begin{align*}
    S &\rightarrow V\textcolor{red}{\textbf{=}}L \\
    L &\rightarrow T\textcolor{red}{\textbf{+}}L \; | \; T \\
    T &\rightarrow R \; | \; RV
\end{align*}
$$

где $R$ — любое регулярное выражение из $L(\mathcal{G})$, а $V$ — любая переменная из $\Delta$ $(\Delta \cap \Sigma = \varnothing)$.

Разбор уравнения происходит в методе *parse* класса *Equation*.

### Решение уравнений

**Определение**\
Пусть $X = f(X)$ — уравнение в алгебре $\mathcal{K}$. Решение $\gamma$ этого уравнения называется *наименьшим*, если для любого другого решения $\gamma'$ выполнено $L(\gamma) \subset L(\gamma')$
 
**Утверждение**\
Пусть $\alpha, \beta \in \mathcal{K}$. Тогда $\alpha^{*}\beta$ является наименьшим решением уравнения $X = \alpha X + \beta$, если, при этом $\varepsilon \notin L(\alpha)$, то единственным.

Нахождение наименьшего решения происходит в методе *solve* класса *Equation*.

### Класс уравнений

In [143]:
def parse_var(tokens):
    if len(tokens) != 0 and re.fullmatch(r'[A-Z]', tokens[0]):
        return tokens.pop(0)

class Equation:
    def __init__(self, var, lin_comb):
        self.var = var
        self.lin_comb = lin_comb
        if '' not in self.lin_comb:
            self.lin_comb[''] = Regex()
    
    def set_parse_var(func):
        global parse_var
        if func == None:
            return
        parse_var = func

    def parse(value):
        global parse_var
        tokens = [token for token in ''.join(value.split())]
        def parse_L():
            if len(tokens) == 0:
                raise Exception('Incorrect equation')
            term = parse_T()
            if len(tokens) == 0:
                return [term]
            if tokens.pop(0) != '+' :
                raise Exception('Incorrect equation')
            return [term] + parse_L()
        def parse_T():
            regex = Regex.parse(tokens)
            var = parse_var(tokens) 
            return (var if var != None else '', regex)
        var = parse_var(tokens)
        if var == None:
            raise Exception('Incorrect equation')
        if len(tokens) == 0 or tokens.pop(0) != '=':
            raise Exception('Incorrect equation')
        terms = parse_L()
        lin_comb = {}
        for term in terms:
            if term[0] in lin_comb:
                lin_comb[term[0]] += term[1]
                continue
            lin_comb[term[0]] = term[1]
        return Equation(var, lin_comb)
    
    def solve(self):
        if self.var not in self.lin_comb:
            return self
        multiplier = Regex.kleene_star(self.lin_comb[self.var])
        self.lin_comb = {var: multiplier * coef for var, coef in self.lin_comb.items()}
        del self.lin_comb[self.var]
        return self

    def substitute(self, equation):
        if equation.var not in self.lin_comb:
            return self
        multipiler = self.lin_comb[equation.var]
        del self.lin_comb[equation.var]
        for var in equation.var:
            if var not in self.lin_comb:
                self.lin_comb[equation.var] = Regex()
        for var in self.lin_comb:
            if var in equation.lin_comb:
                self.lin_comb[var] += multipiler * equation.lin_comb[var]
        return self
    
    def __str__(self):
        result = f'{self.var} ='
        for var, coef in self.lin_comb.items():
            if coef.is_zero():
                continue
            if coef.is_one():
                result += f' {var} +' if var != '' else f' {coef} +'
                continue
            result += f' {coef}{var} +'
        if result[-1] == '=':
            return f'{result} {self.lin_comb[""]}'
        return result[:-2]

## Системы

### Парсер входных данных

Регулярные выражения входных данных импортируются из файла [rules.py](https://github.com/SmEgDm/tfl/tree/main/labs/lab2/regex_system_solver/rules.py).

In [144]:
import rules

def parse(text):
    if not re.fullmatch(rules.system, text):
        raise Exception('Incorrect system')
    system = []
    left_vars = set(['']); right_vars = set()
    for equation in re.findall(rules.equation, text):
        left_vars.add(re.search(rules.var, equation).group(0))
        right_vars = right_vars.union(set([right_var for _, right_var in re.findall(f'({rules.regex})({rules.var}|)', equation)]))
        system.append(equation)
    if right_vars > left_vars:
        raise Exception('Incorrect system')
    return system

### Решение систем

**Определение**\
Решение $(\gamma_1, ...,\gamma_n)$ системы $(1)$ называется наименьшим, если для любого решения $(\gamma_1', ...,\gamma_n')$ системы $(1)$ выполнено: $\forall i=\overline{1,n}: L(\gamma_i) \subset L(\gamma_i')$.

**Алгоритм**\
<u>Прямой ход</u> (по $i$ от $1$ до $n$): в уравнение $X_i = \alpha_{i1}X_1 + ... + \alpha_{in}X_n + \beta_i$ подставляем $X_1, ..., X_{i - 1}$ (если $i = 1$, то пропускаем подстановку), затем находим наименьшее решение у полученного уравнения в предположении, что $X_{i + 1}, ..., X_{n}$ известны.\
В итоге имеем уравнения вида $X_i = \lambda_{i(i+1)}X_{i+1} + ... + \lambda_{in}X_n + \mu_{i}$ (если $i = n$, то $X_n = \mu_n$).

<u>Обратный ход</u> (по $i$ от $n$ до $1$): в уравнение $X_i = \lambda_{i(i+1)}X_{i+1} + ... + \lambda_{in}X_n + \mu_{i}$ подставляем $X_{i+1}, ..., X_n$ (если $i = n$, то пропускаем подстановку).\
Получаем уравнения, в которых отсутствуют переменные в правых частях — это результат работы алгоритма.

**Утверждение**\
Вышеописанный алгоритм находит наименьшее решение системы $(1)$.

In [145]:
def solve_system(equations):
    n = len(equations)
    # Прямой ход
    for i in range(n):
        for j in range(i):
            equations[i].substitute(equations[j])
        equations[i].solve()
    # Обратный ход
    for i in range(n - 1, -1, -1):
        for j  in range(i + 1, n):
            equations[i].substitute(equations[j])
    return equations

## Тесты

### Системы по синтаксису входных данных

In [146]:
import os
from ipywidgets import Tab, HTML

tests_count = len(os.listdir('tests'))

children = []
output_pattern = lambda input_text, result: HTML(
    f'''
    <div>
        <b>Input</b><br>
        {input_text}<br><br>
        <b>Result</b><br>
        {result}
    </div>
    '''
)

for i in range(tests_count):
    with open(f'tests/test_{i}.txt') as f:
        text = f.read()
        try:
            system = [Equation.parse(equation) for equation in parse(text)]
            solve_system(system)
            result = '<br>'.join(map(str, system))
        except Exception as err:
            result = f'<b>{err}</b>'
        children.append(output_pattern('<br>'.join(text.split('\n')), result))

tab = Tab()
for i in range(tests_count):
    tab.set_title(i, f'Test {i + 1}')
tab.children = children

tab

### Системы общего вида

In [147]:
import sys

sys.path.append('other_tests')

tests_count = len(os.listdir('other_tests'))

children = []

for i in range(tests_count):
    with open(f'other_tests/test_{i}.txt') as f:
        text = f.read()
        try:
            system = [Equation.parse(equation) for equation in text.split('\n')]
            solve_system(system)
            result = '<br>'.join(map(str, system))
        except Exception as err:
            result = f'<b>{err}</b>'
        children.append(output_pattern('<br>'.join(text.split('\n')), result))

tab = Tab()
for i in range(tests_count):
    tab.set_title(i, f'Test {i + 1}')
tab.children = children

tab